In [1]:
import asyncio
import inspect
from interrupter.interrupter import Interrupter
from interrupter.cancellable import Cancellable

In [2]:
interrupter = Interrupter(None, events=None, debug=True)

In [3]:
async def run(value: str):
    await asyncio.sleep(2)
    print(f"value: {value}")

asyncio.iscoroutine(run)

False

In [4]:
inspect.iscoroutine( run)

False

In [9]:
class Cancellable:
    # could define the interrupter in the init
    def __init__(self, interrupter: Interrupter):
        self.interrupter = interrupter

    def __call__(self, func, *args, **kwargs):
        print(f"__call__ func: {func}")
        # if not inspect.iscoroutine(func):
            # raise TypeError(f"Cancellable function received MUST be a coroutine --- function received: {func} --- Make sure you have 'called' the coroutine and given it the required arguments. Ex: my_coro('first arg', 'second arg') ")
        self.task = asyncio.create_task(func)
        print(f"__call__ task: {self.task}")
        return self

    async def __aenter__(self):
        print(f"__enter__ func: {self.task}")
        self.interrupter.register_cancellable(self.task)
        return self.task

    async def __aexit__(self, exc_type, exc_value, exc_tb):
        print(f"exc_type: {exc_type} -- exc_value: {exc_value} -- exc_tb: {exc_tb}")
        self.interrupter.deregister_cancellable(self.task)
        print("exiting")

In [10]:
cancellable = Cancellable(interrupter)



In [13]:
async with cancellable(run('hey')) as task:
    print(f'with task: {task}')
    # task.cancel()
    await task


__call__ func: <coroutine object run at 0x000001EA55C9DD80>
__call__ task: <Task pending name='Task-13' coro=<run() running at C:\Users\Damon\AppData\Local\Temp\ipykernel_12216\2920663390.py:1>>
__enter__ func: <Task pending name='Task-13' coro=<run() running at C:\Users\Damon\AppData\Local\Temp\ipykernel_12216\2920663390.py:1>>
Interrupter registering cancellable: <Task pending name='Task-13' coro=<run() running at C:\Users\Damon\AppData\Local\Temp\ipykernel_12216\2920663390.py:1>>
with task: <Task pending name='Task-13' coro=<run() running at C:\Users\Damon\AppData\Local\Temp\ipykernel_12216\2920663390.py:1>>
value: hey
exc_type: None -- exc_value: None -- exc_tb: None
Interrupter deregistering cancellable: <Task finished name='Task-13' coro=<run() done, defined at C:\Users\Damon\AppData\Local\Temp\ipykernel_12216\2920663390.py:1> result=None>
exiting
